# SC3 - Shape-Controlled Clustering and Christofides algorithm

Execution example of the *Shape-Controlled Clustering and Christofides algorithm* (*SC3*).

## Dependencies

In [1]:
import pandas as pd
from haversine import haversine
from utils.utils import get_config_from_yaml, load_data, filter_by_level
from utils.simulation import Simulation
from utils.distances import slender_distance
from routing.googlerouting import GoogleRouting
from clustering.adjustedkmeans import AdjustedKMeans

## Configuration loading

In [2]:
YAML = "configs/example.yaml"

config = get_config_from_yaml(YAML)
depot = [config.depot.latitude, config.depot.longitude]
start_date = pd.to_datetime(config.data.period.start_date)
end_date = pd.to_datetime(config.data.period.end_date)

## Data loading and preprocessing

In [3]:
data = load_data(config.data.file_path)
data = data[config.data.period.start_date : config.data.period.end_date]
data

,bin_serial,bin_level,occluded,outdoor,latitude,longitude
detected_at,,,,,,
2019-09-01 00:00:00,4318008082,1,False,Si,45.482284,9.189676
2019-09-01 00:00:00,4118006723,2,False,Si,45.482619,9.193246
2019-09-01 00:00:00,1219014465,1,False,Si,45.467695,9.206261
2019-09-01 00:00:00,4118007115,2,False,Si,45.486551,9.186808
2019-09-01 00:00:00,1419015438,4,False,Si,45.483160,9.213806
...,...,...,...,...,...,...
2019-09-30 23:59:00,219011641,1,False,Si,45.481596,9.208321
2019-09-30 23:59:00,4318008012,2,False,Si,45.489001,9.187497
2019-09-30 23:59:00,5018010099,3,False,Si,45.480963,9.226391


## Configurations

In [4]:
# Clustering parameters
clustering_kwargs = {
    'max_size': 200,
    'balanced': False, # same number of bins in every cluster?
    'distance': lambda p1, p2: slender_distance(p1, p2, depot, 1, 0), # function to use for clustering creation
}

# Routing parameters
routing_kwargs = {
    'distance_function': haversine,
    'vehicle_capacities': 200,
    'num_vehicles': 1, # routing computed for one cluster at a time
}

# Final configuration
SC3_config = {
    'cluster_class': AdjustedKMeans,
    'cluster_kwargs': clustering_kwargs,
    'graph_class': GoogleRouting,
    'graph_kwargs': routing_kwargs,
}

## Simulation

In [5]:
simulation = Simulation(depot, SC3_config, window_size=6, max_size=200, filter_function=filter_by_level, filter_kwargs={})
routes = simulation.compute_simulation(data, start_date, end_date, speed=30, emp_time=60, debug=False)
simulation.print_simulation()
simulation.to_csv('output/SC3.csv')

# Simulation #
Shifts executed: 120.
Total distance: 17675.593510000002 km.
Bins emptied: 130845.
Vehicles involved: 709.
